* DeepLearning : https://medium.com/@intheblackworld/deep-learning-tutorial-%E5%BF%83%E5%BE%97-b1f7f84a497d
* Fast.ai Lesson Note : https://medium.com/@hiromi_suenaga/deep-learning-2-part-1-lesson-2-eeae2edd2be4
* Fast.ai Lesson Note : https://medium.com/@easonlalala/fast-ai-lesson-1-%E7%AD%86%E8%A8%98-6d696a89b0b0

![Fast.ai 課程結構](https://cdn-images-1.medium.com/max/800/1*D0WqPCX7RfOL47TOEfkzYg.png)

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import os
print(os.listdir("../input/newdogscats/dogscats/dogscats"))
PATH = "../input/newdogscats/dogscats/dogscats/"
# Any results you write to the current directory are saved as output.

In [ ]:
os.listdir(PATH)

In [ ]:
torch.cuda.is_available()

In [ ]:
torch.backends.cudnn.enabled


In [ ]:
os.listdir(f'{PATH}valid')

In [ ]:
files = os.listdir(f'{PATH}valid/cats')[:5]
files

In [ ]:
img = plt.imread(f'{PATH}valid/cats/{files[0]}')
plt.imshow(img);

In [ ]:
arch=resnet34
sz=224
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz))
TMP_PATH = "/tmp/tmp"
MODEL_PATH = "/tmp/model"
learn = ConvLearner.pretrained(arch, data, precompute=True, tmp_name=TMP_PATH, models_name=MODEL_PATH)

* Resnet34，全名是Residual Network with 34 layers，模型本身透過1.3億張含有2萬種標籤的照片訓練後得到CNN模型
* sz代表照片尺寸，理論上模型可以使用任何尺寸的照片重新縮放後進行訓練
* 因為Resnet訓練用的照片大多是224x224 or 299x299，因此我們把所有照片縮放到224x224 pixels 來訓練
* ImageClassifierData.from_paths讀取資料夾內的訓練照片，tfms_from_model這個函式表示說我們如何把照片轉成pre-trained model可以取用的規格
* ConvLearner.pretrained 把資料丟到Resnet 中，並且將加入符合訓練資料的輸出層，並且開啟precompute

In [ ]:
lrf=learn.lr_find()

[Learning rate](https://itw01.com/VUYJLEX.html)
* Gradient Descent
* GD我們是一次用全部訓練極的數據去計算損失函數的梯度就更新一次參數
* Stochastic Gradient Descent
* SGD就是一次跑一個樣本或是小批次(mini-batch)樣本然後算出一次梯度或是小批次梯度的平均後就更新一次，那這個樣本或是小批次的樣本是隨機抽取
![Gradient Descent](https://cdn-images-1.medium.com/max/800/1*cWCG1LQgGZ268J9tdlyOpA.png)![Learning rate](https://cdn-images-1.medium.com/max/400/1*X5QbtfhszYNBSV6DZzbT7A.jpeg)

![Cyclical Learning Rates (CLR)](https://img.itw01.com/images/2018/03/19/10/3454_t2xA7Z_VUYJLEX.jpg!r1024x0.jpg)
* Cyclical Learning Rates （CLR)
* CLR希望透過可變的LR來增加訓練的準確度。通常我們希望隨著訓練過程越來越接近最低點，LR隨著越小越好，這樣一來可以很快找到最佳化的方向，並且兼顧在最低點附近微調。
1.  設定最大學習率, Maximum Learning Rate, MLR
2. 根據特定函數逐步減少Learning Rate，稱為LR Annealing
![Cosine](https://github.com/apachecn/fastai-ml-dl-notes-zh/raw/master/img/1_xmIlOee7PWLc6fa7xdfRkA.png)
3. 每完成一個循環Cycle，重啟Learning Rate(SGD with restart)
![CLR](https://cdn-images-1.medium.com/max/600/1*roAnB_YqpfqHttLgF4tj4Q.png)

In [ ]:
learn.sched.plot_lr()

In [ ]:
learn.sched.plot()

In [ ]:
learn.fit(1e-2, 1)

* 最後learn.fit給定Learning Rate = 0.01 與Learning Cycles (Epochs) = 3 這兩個參數後就可以開始訓練了。
* Learning Rate代表在Gradient Descent 中每一步的大小，Learning Cycles則是代表我們要讓模型學習所有的資料多少次。

[Epoch Batch Iteration](https://kknews.cc/zh-tw/other/r8v66nx.html)
* 每個 Epoch 要訓練的圖片數量： 5000
* 訓練集具有的 Batch 個數： 5000 /256 = 195 + 1 = 196
* 每個 Epoch 需要完成的 Batch 個數： 196
* 每個 Epoch 具有的 Iteration 個數： 196
* 每個 Epoch 中發生模型權重更新的次數： 196
* 訓練 10 Epochs，模型權重更新的次數： 196 * 10

* fast.ai使用的是Cosine函數，讓LR隨著訓練過程逐步減少。每一個Cycle都從MLR逐步變小
* 直到一個Cycle結束後又重回最大值重新訓練，也就是restart

In [ ]:
learn.fit(1e-2, 3, cycle_len=1)

* Underfitting，因為參數過少連Training set都會有頗大的預測誤差
* Overfitting，因為參數過多導致過度符合Training set的資料特性，使得其無法預測較為普遍的資料集

In [ ]:
learn.sched.plot_lr()

* Data Augmentation
* 在訓練模型時資料的大小可以說是最關鍵的因素之一。
* 為了使有限的資料達到更精確的模型我們使用Data Augmentation來增加Training Set的大小。
* 原則上各種轉換要能夠保留訓練目標的特徵，常見的轉換有鏡像轉換Mirror、翻轉Flip、縮放Scale、隨機剪裁Random Crop、旋轉Rotation 或是扭曲Shearing。
* Data Augmentation又可以分成Train Time Data Augmentation 與 Test Time Data Augmentation。
* 前者在訓練時加入擴增影像來增加資料大小，提高模型準確度。後者則是在測試階段將測試樣本轉換成數個影像讓模型判別再取平均，降低誤判的機率。

In [ ]:
tfms = tfms_from_model(resnet34, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms)
learn = ConvLearner.pretrained(arch, data, precompute=True, tmp_name=TMP_PATH, models_name=MODEL_PATH)
learn.fit(1e-2, 1)

* Now we created a new data object that includes augmentation. Initially, the augmentations actually do nothing because of precompute=True.
* We are using a pre-trained network which has already learned to recognize features (i.e. we do not want to change hyper parameters it learned), so what we can do is to pre-compute activations for hidden layers and just train the final linear portion.
* To use data augmentation, we have to do learn.precompute=False

In [ ]:
learn.precompute=False
learn.fit(1e-2, 3, cycle_len=1)

In [ ]:
learn.sched.plot_lr()

Saving model

In [ ]:
learn.save('224_lastlayer')

In [ ]:
print(os.listdir("/tmp/"))

In [ ]:
print(os.listdir("/tmp/tmp"))

In [ ]:
print(os.listdir("/tmp/model"))

In [ ]:
learn.load('224_lastlayer')

Fine Tuning and Differential Learning Rate
* Pre-trained model跟我們的訓練的目標有高度的相似性，例如說狗或貓。假使是衛星空拍影像或是病理切片這種可能跟ImageNet中沒那麼相似的話訓練效果可能就會大打折扣
* ConvLearner.pretrained 函式預設在Pre-trained model的最後加入兩層Dense Neuron Net使得Output符合我們要訓練的目標。
* 並且預設trainable weight只有包含最後那兩層，換句話說前面所有的weight都是frozen不會隨著訓練做改變。
* learn.unfreeze() 可以釋放所有先前pretrained weight重新調整、訓練。在架構不變的情況下，微調其中的參數可以讓transfer model 的準確度大大提升。在unfreeze的情況下，所有模型中的參數都會隨著訓練集而改變。
* learn.unfreeze(n) 可以輸入你想要unfreeze的layer

In [ ]:
learn.summary()

In [ ]:
learn.unfreeze()

In [ ]:
learn.summary()

* Earlier we said 3 is the number of epochs, but it is actually cycles. 
* So if cycle_len=2 , it will do 3 cycles where each cycle is 2 epochs (i.e. 6 epochs). 
* cycle_mult=2 : this multiplies the length of the cycle after each cycle (1 epoch + 2 epochs + 4 epochs = 7 epochs).

In [ ]:
lr=np.array([1e-4,1e-3,1e-2])
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.sched.plot_lr()

In [ ]:
learn.save('224_all')

In [ ]:
learn.load('224_all')

Test Time Data Augmentation
* What this means is that we are going to take 4 data augmentations at random as well as the un-augmented original (center-cropped). 
* We will then calculate predictions for all these images, take the average, and make that our final prediction. 
* Note that this is only for validation set and/or test set.

In [ ]:
log_preds,y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)

In [ ]:
accuracy_np(probs, y)

In [ ]:
preds = np.argmax(probs, axis=1)
probs = probs[:,1]

Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, preds)

In [ ]:
plot_confusion_matrix(cm, data.classes)

1. Use lr_find() to find highest learning rate where loss is still clearly improving
1. Train last layer with data augmentation (i.e. precompute=False) for 2–3 epochs with cycle_len=1
1. Unfreeze all layers
1. Set earlier layers to 3x-10x lower learning rate than next higher layer. 
1. Train full network with cycle_mult=2 until over-fitting